In [46]:
shape_cat="mcal"
vol="vlim"
z_range="all_z"
key=""
quantile=""
test=""
z_diff=""

In [47]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import fitsio as fio
import numpy as np
import ast

import astropy.cosmology
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from IPython.display import display, Math
from tqdm.notebook import tqdm
tqdm.pandas()

/global/homes/z/zchusre/.conda/envs/desdesi/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [48]:
#This header file contains all the common packages
%matplotlib inline
%run -i /global/project/projectdirs/des/zhou/sa/lib/src/header.py $shape_cat $vol $z_range 
%run -i /global/project/projectdirs/des/zhou/sa/lib/src/func.py  

In [ ]:
BPZ_table=Table(fits.open('/global/project/projectdirs/des/zhou/y1a1_data/mcal-y1a1-combined-griz-blind-v3-matched_BPZbase.fits')[1].data)

In [ ]:
BPZ_table

In [ ]:
BPZ_table=BPZ_table[['mean_z','coadd_objects_id']]

In [ ]:
BPZ_half=BPZ_table.to_pandas()
BPZ_table=None

In [ ]:
mcal = fio.FITS('/global/project/projectdirs/des/zhou/y1a1_data/mcal-y1a1-combined-riz-unblind-v4-matched.fits',memmap=True,mode='r')[1]
mcal_id=mcal['coadd_objects_id'][:] 

In [ ]:
mcal

In [ ]:
BPZ_half.set_index("coadd_objects_id",inplace=True)

In [ ]:
BPZ_id=BPZ_half.index.to_numpy()
shapes_id=shapes.index.to_numpy()

In [ ]:
i_mcal,i_BPZ=sort2n(mcal_id,BPZ_id)

In [ ]:
mcal_half=pd.DataFrame(mcal.read(rows=[i_mcal],columns=['coadd_objects_id','ra','dec','e1','e2','flags_select']).byteswap().newbyteorder())
mcal_columns=mcal_half.columns
mcal_product_col=pd.MultiIndex.from_product([['All'],mcal_columns],names=['Band','Data'])
mcal_half.columns=mcal_product_col
mcal_half[('All','RA')]=mcal_half[('All','ra')]
mcal_half[('All','DEC')]=mcal_half[('All','dec')]
mcal_half.drop(columns=[('All','ra'),('All','dec')],inplace=True)
mcal_half.set_index(('All','coadd_objects_id'),inplace=True)

In [ ]:
BPZ_columns=BPZ_half.columns.to_list()
BPZ_product_col=pd.MultiIndex.from_product([['All'],BPZ_columns],names=['Band','Data'])
BPZ_half.columns=BPZ_product_col

In [ ]:
drop_columns=[('All',column) for column in BPZ_columns if column!='mean_z']
BPZ_half.drop(columns=drop_columns,inplace=True)

In [ ]:
assert len(BPZ_half)==len(mcal_half)

In [ ]:
mcal_half.sort_index(inplace=True)
BPZ_half.sort_index(inplace=True)

In [ ]:
assert np.all(mcal_half.index==BPZ_half.index)
mcal_half[('All','mean_z')]=BPZ_half[('All','mean_z')]

In [ ]:
BPZ=mcal_half
BPZ.index.name="ID"

In [ ]:
BPZ_half=None
mcal_half=None

# BPZ&shape

In [ ]:
i_BPZ,i_shapes=sort2n(BPZ_id,shapes_id)

In [ ]:
help(sort2n)

In [ ]:
BPZ_half=BPZ.iloc[i_BPZ]

In [ ]:
BPZ_half.drop([('All',           'RA'),
            ('All',          'DEC'),
            ('All',           'e1'),
            ('All',           'e2'),
            ('All', 'flags_select')],inplace=True,axis=1)

In [ ]:
shape_BPZ=pd.concat([shapes,BPZ_half],axis=1,verify_integrity=True)

In [ ]:
# This looks like a suspicious criterion to select memebers

members=shape_BPZ[(shape_BPZ[('All','Z')]-shape_BPZ[('All','mean_z')])<=-0.1]

In [ ]:
shape_BPZ.columns

In [ ]:
len(members)/len(BPZ)

In [ ]:
BINS=np.arange(0,2,0.1)
plt.hist(shape_BPZ[('All','Z')],histtype="step",label="redmapper",bins=BINS)
plt.hist(shape_BPZ[('All','mean_z')],histtype="step",label="BPZ",bins=BINS)
plt.legend()

In [ ]:
plt.scatter(shape_BPZ[('All','Z')],shape_BPZ[('All','mean_z')],s=1,)
# plt.axis('equal')
plt.xlim(0,3)
plt.ylim(0,3)
plt.xlabel("red_z")
plt.ylabel("BPZ")

In [ ]:
len(shape_BPZ[('All','mean_z')])

In [ ]:
shapes[('All','ZRED2')].describe()

# Masking &saving

In [ ]:
BPZ=BPZ[BPZ['All','flags_select']==0]
BPZ.dropna(axis=0,how='any',inplace=True)

In [ ]:
np.any(np.isnan(BPZ[('All','mean_z')]))

In [ ]:
H0=100 #km s-1 Mpc-1
Om0=0.3
Ode0=0.7
lcdm=astropy.cosmology.FlatLambdaCDM(H0, Om0)

In [ ]:
BPZ[('All','angR')]=BPZ.progress_apply(lambda x:np.float(lcdm.angular_diameter_distance(x[('All','mean_z')])/u.Mpc),axis=1)

In [ ]:
shape_BPZ.to_pickle("/global/project/projectdirs/des/zhou/sa/shape_BPZ.pkl")
BPZ.to_pickle("/global/project/projectdirs/des/zhou/sa/BPZ.pkl")